
# 📊 Cryptocurrency Price Analysis with Storytelling

In this notebook, we perform a deep-dive into the behavior of top cryptocurrencies using 2-hour interval data.  
Our goal is to **extract meaningful insights**, identify short-term trends, and **highlight anomalies** that might inform trading or investment decisions.

We'll combine **narrative**, **visuals**, and **metrics** to tell a compelling data story.



## 🔄 Data Collection via CoinGecko API

We fetch data for the top cryptocurrencies every 2 hours using the CoinGecko API. The fields include:

- Name
- Price (USD)
- Market Cap
- 24H Volume
- Circulating Supply
- Timestamp



## 📥 Load and Preview Dataset

We load the previously scraped data (`all_crypto_data.csv`) and begin initial preprocessing.



## 🧹 Data Cleaning

Let’s convert timestamps, handle missing or infinite values, and ensure the dataset is analysis-ready.



# 📈 Exploratory Data Analysis

Now we explore price trends, volume distributions, and compare market caps across coins.

We'll also investigate how much prices changed in the last 2 hours.



## 🔍 Key Insights

- **BTC and ETH maintain price dominance**, but smaller coins show more volatility over 2-hour windows.
- **24H volume spikes** correlate with sharp price changes—indicative of speculative trading.
- **Market cap** concentration remains high, with BTC commanding over 40% in most snapshots.

> These insights can guide short-term trading or alert us to potential pump/dump cycles.



## ✅ Summary & Takeaways

- We successfully pulled and processed 2-hour interval crypto data.
- Using clean and structured analysis, we visualized and explained:
  - Price trends and anomalies
  - Volume surges and volatility
  - Relative performance across coins

**Next Steps:**
- Build a Streamlit dashboard for real-time monitoring.
- Add anomaly detection models to flag sudden movements.
